In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
path = 'C:\\Users\\Administrator\\Desktop\\python\\Recommendation System\\'

In [149]:
ratings = pd.read_csv('ratings.dat'  , sep = '::' , engine = 'python' , header = None )
movies = pd.read_csv('movies.dat'  , sep = '::' , engine = 'python' , header = None )
users = pd.read_csv('users.dat' , sep = '::' , engine = 'python' , header = None)

In [150]:
ratings.head(1)

,0,1,2,3
0,1,1193,5,978300760


In [151]:
movies.head(1)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy


In [152]:
users.head(1)

,0,1,2,3,4
0,1,F,1,10,48067


In [153]:
training_set = pd.read_csv('u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [154]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [155]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [156]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [157]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [158]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nv, nh)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W)
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W.t())
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [159]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [160]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3315)
epoch: 2 loss: tensor(0.2463)
epoch: 3 loss: tensor(0.2506)
epoch: 4 loss: tensor(0.2495)
epoch: 5 loss: tensor(0.2469)
epoch: 6 loss: tensor(0.2472)
epoch: 7 loss: tensor(0.2458)
epoch: 8 loss: tensor(0.2460)
epoch: 9 loss: tensor(0.2463)
epoch: 10 loss: tensor(0.2474)


In [162]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
    print('test loss: '+str(test_loss/s))

test loss: tensor(0.2426)
test loss: tensor(0.2804)
test loss: tensor(0.2767)
test loss: tensor(0.2575)
test loss: tensor(0.2703)
test loss: tensor(0.2566)
test loss: tensor(0.2470)
test loss: tensor(0.2377)
test loss: tensor(0.2446)
test loss: tensor(0.2335)
test loss: tensor(0.2350)
test loss: tensor(0.2250)
test loss: tensor(0.2294)
test loss: tensor(0.2255)
test loss: tensor(0.2332)
test loss: tensor(0.2265)
test loss: tensor(0.2394)
test loss: tensor(0.2317)
test loss: tensor(0.2300)
test loss: tensor(0.2345)
test loss: tensor(0.2414)
test loss: tensor(0.2438)
test loss: tensor(0.2394)
test loss: tensor(0.2371)
test loss: tensor(0.2287)
test loss: tensor(0.2321)
test loss: tensor(0.2334)
test loss: tensor(0.2286)
test loss: tensor(0.2329)
test loss: tensor(0.2400)
test loss: tensor(0.2422)
test loss: tensor(0.2461)
test loss: tensor(0.2477)
test loss: tensor(0.2522)
test loss: tensor(0.2557)
test loss: tensor(0.2663)
test loss: tensor(0.2632)
test loss: tensor(0.2653)
test loss: t

test loss: tensor(0.2557)
test loss: tensor(0.2556)
test loss: tensor(0.2561)
test loss: tensor(0.2557)
test loss: tensor(0.2552)
test loss: tensor(0.2552)
test loss: tensor(0.2551)
test loss: tensor(0.2551)
test loss: tensor(0.2546)
test loss: tensor(0.2540)
test loss: tensor(0.2549)
test loss: tensor(0.2557)
test loss: tensor(0.2555)
test loss: tensor(0.2555)
test loss: tensor(0.2561)
test loss: tensor(0.2564)
test loss: tensor(0.2572)
test loss: tensor(0.2569)
test loss: tensor(0.2563)
test loss: tensor(0.2561)
test loss: tensor(0.2555)
test loss: tensor(0.2549)
test loss: tensor(0.2550)
test loss: tensor(0.2551)
test loss: tensor(0.2545)
test loss: tensor(0.2539)
test loss: tensor(0.2533)
test loss: tensor(0.2527)
test loss: tensor(0.2528)
test loss: tensor(0.2522)
test loss: tensor(0.2516)
test loss: tensor(0.2522)
test loss: tensor(0.2519)
test loss: tensor(0.2520)
test loss: tensor(0.2521)
test loss: tensor(0.2520)
test loss: tensor(0.2514)
test loss: tensor(0.2516)
test loss: t

test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: tensor(0.2550)
test loss: t

In [163]:
l = [2,0,0,-2]

In [166]:
for i in l :
    print(i)
    print( np.exp(i) / sum( [np.exp(i) for i in l] ) )

2
0.775803492574376
0
0.10499358540350652
0
0.10499358540350652
-2
0.01420933661861104
